# Построение baseline-решений
В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:
1. Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
2. Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
3. Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
4. Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:
1. Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
2. Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
3. Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)

В качестве решения приложите получившийся jupyther notebook. Убедитесь, что в нем присутствуют:
- все baseline-модели, которые вы построили;
- качество всех построенных моделей оценено с помощью кросс-валидации, и это понятно из текста в jupyther notebook;
- все модели оценены с помощью основной и дополнительных метрик качества.

In [83]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import RidgeClassifier
from scipy.sparse import coo_matrix, hstack
seed = 1903

Загрузим train dataset, выделенный на предыдущей неделе

In [2]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",")
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv")
print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(27999, 230)
(27999, 1)


## Построение линейной baseline модели

In [3]:
X_ridge = churn_data_frame.as_matrix()
y_ridge = churn_labels_frame.as_matrix().flatten()
categorial_columns_indices = range(190, 230-190)

In [6]:
X_categorial = X_ridge[:, 190:]

In [77]:
def one_hot_encode(X_categorial):
    X_num = np.empty(X_categorial.shape)
    for column_number in range(X_categorial.shape[1]):
        labelEncoder = LabelEncoder()
        column = X_categorial[:,column_number]
        for idx, val in enumerate(column):
            if(not(isinstance(val, str))):
                column[idx] = "NaV"
        num_column = labelEncoder.fit_transform(column)
        for row_number, val in enumerate(num_column):
            X_num[row_number, column_number] = val
    one_hot = OneHotEncoder()
    return one_hot.fit_transform(X_num)

In [85]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1903)
train_indices, test_indices = list(skf.split(X_ridge, y_ridge))[0]
X_train = X_ridge[train_indices]
X_test = X_ridge[test_indices]
y_train = y_ridge[train_indices]
y_test = y_ridge[test_indices]
X_numeric = X_train[:,:190]
X_categorial = X_train[:,190:]
encoded = one_hot_encode(X_categorial)
print(X_train.shape, X_numeric.shape, X_categorial.shape, encoded.shape)
#hstack([coo_matrix(X_numeric), encoded])
# ridge = RidgeClassifier()
# ridge.fit(X_train, y_train)

(18665, 230) (18665, 190) (18665, 40) (18665, 40571)
[[nan nan nan ..., nan 312.0 nan]
 [nan nan nan ..., nan nan nan]
 [nan nan nan ..., nan nan nan]
 ..., 
 [nan nan nan ..., nan nan nan]
 [nan nan nan ..., nan nan nan]
 [nan nan nan ..., nan nan nan]]


In [ ]:
# До обработки данных заполнить nan значения для категориальных и числовых признаков.